# Aula 1 - Buscando a similaridade

In [57]:
url = 'https://raw.githubusercontent.com/Mirlaa/NLP-trabalhando-similaridade-sentencas/refs/heads/main/reviews_zoop.csv'

In [58]:
import pandas as pd

dados = pd.read_csv(url)
dados

,nota_review,review
0,5,Recebi bem antes do prazo estipulado.
1,5,Parabéns lojas Zoop adorei comprar pela Intern...
2,4,aparelho eficiente. no site a marca do aparelh...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,5,"Vendedor confiável, produto ok e entrega antes..."
...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...
39458,3,"O produto não foi enviado com NF, não existe v..."
39459,5,"Excelente mochila, entrega super rápida. Super..."
39460,1,Solicitei a compra de uma capa de retrovisor c...


In [59]:
dados['review'][1]

'Parabéns lojas Zoop adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa'

In [60]:
dados['review'][8]

'A compra foi realizada facilmente.\r\nA entrega foi efetuada muito antes do prazo dado.\r\nO produto já começou a ser usado e até o presente,\r\nsem problemas.'

In [61]:
dados['review'][11]

'Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No  o prazo máximo já se esgotou e ainda não recebi o produto.'

In [62]:
dados['review'][18]

'otimo vendedor chegou ate antes do prazo , adorei o produto'

## Similaridade entre as palavras

In [63]:
from nltk.tokenize import word_tokenize
from functools import partial

word_tokenize_pt = partial(word_tokenize, language = 'portuguese')

In [64]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [65]:
!pip install gensim

In [66]:
from gensim.models import Word2Vec

modelo = Word2Vec(sentences = dados['review'].apply(word_tokenize_pt),
                  vector_size=100, min_count=1, window=5,
                  workers=1, seed=45)

In [67]:
modelo.wv.most_similar('entrega')

[('Entrega', 0.7579216361045837),
 ('entregue', 0.7184001207351685),
 ('compra', 0.7105124592781067),
 ('chegou', 0.6817407608032227),
 ('entregou', 0.6612473130226135),
 ('preparar', 0.6325778365135193),
 ('envio', 0.6313653588294983),
 ('sorriso', 0.6210829615592957),
 ('Chegou', 0.6198225021362305),
 ('mercadoria', 0.6146730780601501)]

In [68]:
modelo.wv.most_similar('vendedor')

[('atendimento', 0.8572612404823303),
 ('serviço', 0.8074656128883362),
 ('fornecedor', 0.7500720024108887),
 ('veiculo', 0.7239760756492615),
 ('produto', 0.7152891755104065),
 ('processo', 0.7085978388786316),
 ('parceiro', 0.7057496905326843),
 ('cumpriu', 0.6909526586532593),
 ('trabalho', 0.6848700642585754),
 ('cumpriram', 0.6713030934333801)]

In [69]:
modelo.wv.most_similar('produto')

[('pedido', 0.749972939491272),
 ('relógio', 0.7460570335388184),
 ('vendedor', 0.7152891755104065),
 ('endereço', 0.7050330638885498),
 ('Produto', 0.6978251934051514),
 ('mesmo', 0.6896668076515198),
 ('tempo', 0.6797294616699219),
 ('Mas', 0.6640905737876892),
 ('mercadoria', 0.6585670113563538),
 ('condizia', 0.6553379893302917)]

## Similaridade entre sentenças

In [70]:
dados['review_token'] = dados['review'].apply(word_tokenize_pt)

In [71]:
from gensim.models.doc2vec import TaggedDocument

dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]

In [72]:
dados_tag

[TaggedDocument(words=['Recebi', 'bem', 'antes', 'do', 'prazo', 'estipulado', '.'], tags=['0']),
 TaggedDocument(words=['Parabéns', 'lojas', 'Zoop', 'adorei', 'comprar', 'pela', 'Internet', 'seguro', 'e', 'prático', 'Parabéns', 'a', 'todos', 'feliz', 'Páscoa'], tags=['1']),
 TaggedDocument(words=['aparelho', 'eficiente', '.', 'no', 'site', 'a', 'marca', 'do', 'aparelho', 'esta', 'impresso', 'como', '3desinfector', 'e', 'ao', 'chegar', 'esta', 'com', 'outro', 'nome', '...', 'atualizar', 'com', 'a', 'marca', 'correta', 'uma', 'vez', 'que', 'é', 'o', 'mesmo', 'aparelho'], tags=['2']),
 TaggedDocument(words=['Mas', 'um', 'pouco', ',', 'travando', '...', 'pelo', 'valor', 'ta', 'Boa', '.'], tags=['3']),
 TaggedDocument(words=['Vendedor', 'confiável', ',', 'produto', 'ok', 'e', 'entrega', 'antes', 'do', 'prazo', '.'], tags=['4']),
 TaggedDocument(words=['GOSTARIA', 'DE', 'SABER', 'O', 'QUE', 'HOUVE', ',', 'SEMPRE', 'RECEBI', 'E', 'ESSA', 'COMPRA', 'AGORA', 'ME', 'DECPCIONOU'], tags=['5']),
 T

In [73]:
from gensim.models import Doc2Vec

modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)

In [74]:
vetor_inferido = modelo.infer_vector(['entrega'])
vetor_inferido

array([-0.10697037, -0.06737058,  0.06194549, -0.05522205,  0.08310623,
        0.05488851, -0.03042652, -0.02161958,  0.01454077, -0.09969404,
       -0.11583801,  0.03037656, -0.09264313,  0.03699465, -0.07300633,
       -0.06591807, -0.05618995,  0.01700832, -0.03732764,  0.02003834,
        0.02724007, -0.0251933 ,  0.08816136, -0.02152258, -0.01954172,
       -0.00055314, -0.09674083, -0.07849121, -0.04488697, -0.0432848 ,
        0.06344956, -0.02174011,  0.02393263,  0.11285095,  0.04102076,
       -0.03453576, -0.10905944,  0.14447051, -0.10860889,  0.04501934,
        0.01401055,  0.05863192,  0.00691673, -0.00672392, -0.01354548,
       -0.10201352,  0.0378913 ,  0.0018659 , -0.04043126, -0.00254339,
        0.00478652, -0.02428935,  0.03168711, -0.02884271,  0.09891071,
       -0.11785528, -0.03006832, -0.02451764, -0.04760823, -0.07366647,
        0.02165102, -0.03796098,  0.0164222 , -0.06866538,  0.01904496,
       -0.01154757,  0.05957567,  0.00259975,  0.02838435,  0.02

In [75]:
frases_similares = modelo.dv.most_similar([vetor_inferido],topn=5)
frases_similares

[('24884', 0.8600223660469055),
 ('24666', 0.8465315699577332),
 ('9304', 0.8419812321662903),
 ('33496', 0.8341566324234009),
 ('30445', 0.8292081356048584)]

In [76]:
for idx, similaridade in frases_similares:
    print(f"Review: {dados.iloc[int(idx)]['review']} - Similaridade: {similaridade:.4f}")

Review: entrega antes do prazo - Similaridade: 0.8600
Review: entrega antes do prazo - Similaridade: 0.8465
Review: entrega no prazo. - Similaridade: 0.8420
Review: Ainda estou esperando a entrega!  - Similaridade: 0.8342
Review: Comprem nas Zoop entrega certaaaa - Similaridade: 0.8292


# Aula 2 - Ajustando os textos de review

In [77]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [78]:
import re
from nltk.corpus import stopwords

def tratamento_inicial (texto):
  texto = re.sub(r'\W',' ',texto.lower())

  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')

  return ' '.join([w for w in tokens if w not in stop_words])

In [79]:
dados['tratamento_1'] = dados['review'].apply(tratamento_inicial)

In [80]:
dados['tratamento_1']

,tratamento_1
0,recebi bem antes prazo estipulado
1,parabéns lojas zoop adorei comprar internet se...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiável produto ok entrega antes prazo
...,...
39457,entregou dentro prazo produto chegou condições...
39458,produto não enviado nf não existe venda nf cer...
39459,excelente mochila entrega super rápida super r...
39460,solicitei compra capa retrovisor celta prisma ...


In [81]:
dados['review'][11]

'Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No  o prazo máximo já se esgotou e ainda não recebi o produto.'

In [82]:
dados['tratamento_1'][11]

'sempre compro internet entrega ocorre antes prazo combinado acredito prazo máximo prazo máximo esgotou ainda não recebi produto'

## Uniformizando o texto

In [83]:
!pip install unidecode

In [84]:
import unidecode

sem_acentos = [unidecode.unidecode(texto) for texto in dados['tratamento_1']]

In [85]:
dados['tratamento_2'] = sem_acentos
dados['tratamento_2']

,tratamento_2
0,recebi bem antes prazo estipulado
1,parabens lojas zoop adorei comprar internet se...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiavel produto ok entrega antes prazo
...,...
39457,entregou dentro prazo produto chegou condicoes...
39458,produto nao enviado nf nao existe venda nf cer...
39459,excelente mochila entrega super rapida super r...
39460,solicitei compra capa retrovisor celta prisma ...


In [86]:
dados['review_token'] = dados['tratamento_2'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])],topn=5)
for idx, similaridade in frases_similares:
    print(f"Review: {dados.iloc[int(idx)]['tratamento_2']} - Similaridade: {similaridade:.4f}")

Review: recebi produto antes prazo entrega intercorrencias - Similaridade: 0.8712
Review: entrega antes prazo - Similaridade: 0.8706
Review: entrega prazo produto acordo anuncio - Similaridade: 0.8696
Review: prazo entrega gigante assim conseguiram atrasar entrega - Similaridade: 0.8696
Review: entrega prazo produto conforme solicitado - Similaridade: 0.8636


## Correções nos dados

In [87]:
df = dados.drop_duplicates(subset= ['tratamento_2'])
df

,nota_review,review,review_token,tratamento_1,tratamento_2
0,5,Recebi bem antes do prazo estipulado.,"[recebi, bem, antes, prazo, estipulado]",recebi bem antes prazo estipulado,recebi bem antes prazo estipulado
1,5,Parabéns lojas Zoop adorei comprar pela Intern...,"[parabens, lojas, zoop, adorei, comprar, inter...",parabéns lojas zoop adorei comprar internet se...,parabens lojas zoop adorei comprar internet se...
2,4,aparelho eficiente. no site a marca do aparelh...,"[aparelho, eficiente, site, marca, aparelho, i...",aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n","[pouco, travando, valor, ta, boa]",pouco travando valor ta boa,pouco travando valor ta boa
4,5,"Vendedor confiável, produto ok e entrega antes...","[vendedor, confiavel, produto, ok, entrega, an...",vendedor confiável produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo
...,...,...,...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...,"[entregou, dentro, prazo, produto, chegou, con...",entregou dentro prazo produto chegou condições...,entregou dentro prazo produto chegou condicoes...
39458,3,"O produto não foi enviado com NF, não existe v...","[produto, nao, enviado, nf, nao, existe, venda...",produto não enviado nf não existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...
39459,5,"Excelente mochila, entrega super rápida. Super...","[excelente, mochila, entrega, super, rapida, s...",excelente mochila entrega super rápida super r...,excelente mochila entrega super rapida super r...
39460,1,Solicitei a compra de uma capa de retrovisor c...,"[solicitei, compra, capa, retrovisor, celta, p...",solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...


In [88]:
df['review'][120]

'A'

In [89]:
df['tratamento_2'][120]

''

In [90]:
df = df[df['tratamento_2'] != '']

In [91]:
df['tratamento_2'][3319]

'simplismeente ameiii'

In [92]:
df['tratamento_2'][8354]

'recebi rapido antes prazo otimo qualidade adoreii'

In [93]:
df['tratamento_2'][12843]

'produto chegou direitinho antes muuuito antes prazo superou expectativas'

In [94]:
def normalizar_repeticoes(texto):
  return re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto)

df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)

/tmp/ipython-input-101985271.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)


In [95]:
df['tratamento_3'][3319]

'simplismente amei'

In [96]:
df['tratamento_3'][8354]

'recebi rapido antes prazo otimo qualidade adorei'

In [97]:
df.reset_index(drop=True, inplace=True)

## Lematização

In [98]:
!pip install stanza
!python -m stanza.download pt

/usr/bin/python3: No module named stanza.download


In [99]:
import stanza

stanza.download('pt')
nlp = stanza.Pipeline('pt',
                      processors='tokenize,lemma',
                      use_gpu=False,
                      batch_size= 64,
                      n_process = 4)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: pt (Portuguese) ...
INFO:stanza:File exists: /root/stanza_resources/pt/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: pt (Portuguese):
| Processor | Package         |
-------------------------------
| tokenize  | bosque          |
| mwt       | bosque          |
| lemma     | bosque_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [100]:
def lematizar_texto(textos):
  texto_lematizados = []

  for texto in textos:
    doc_frase = nlp(texto)
    frase_lematizada = ' '.join([palavra.lemma for frase in doc_frase.sentences for palavra in frase.words])
    texto_lematizados.append(frase_lematizada)

  return texto_lematizados

In [101]:
textos = ['gostei muito experiencia comprar',
          'minha filha gostou produto',
          'compra foi facil compra rapida']
lematizar_texto(textos)

['gostar muito experiencia comprar',
 'meu filha gostar produto',
 'compra ser facil compra rapida']

In [102]:
df['tratamento_4'] = lematizar_texto(df['tratamento_3'])

/tmp/ipython-input-503408071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tratamento_4'] = lematizar_texto(df['tratamento_3'])


In [103]:
df['tratamento_4']

,tratamento_4
0,recebi bem antes prazo estipulado
1,parabem loja zop adorei comprar internet segur...
2,aparelho eficiente site marca aparelho imprimi...
3,pouco travar valor to bom
4,vendedor confiavel produto ok entregar antes p...
...,...
31469,entregar dentro prazo produto chegar condicoe ...
31470,produto nao enviado em f nao existir venda em ...
31471,excelente mochila entregar super rapida super ...
31472,solicitei compra capa retrovisor celta prisma ...


# Aula 3 - Otimizando o Doc2vec

In [104]:
df_lem = df.drop_duplicates(subset=['tratamento_4'])
df_lem

,nota_review,review,review_token,tratamento_1,tratamento_2,tratamento_3,tratamento_4
0,5,Recebi bem antes do prazo estipulado.,"[recebi, bem, antes, prazo, estipulado]",recebi bem antes prazo estipulado,recebi bem antes prazo estipulado,recebi bem antes prazo estipulado,recebi bem antes prazo estipulado
1,5,Parabéns lojas Zoop adorei comprar pela Intern...,"[parabens, lojas, zoop, adorei, comprar, inter...",parabéns lojas zoop adorei comprar internet se...,parabens lojas zoop adorei comprar internet se...,parabens lojas zop adorei comprar internet seg...,parabem loja zop adorei comprar internet segur...
2,4,aparelho eficiente. no site a marca do aparelh...,"[aparelho, eficiente, site, marca, aparelho, i...",aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho imprimi...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n","[pouco, travando, valor, ta, boa]",pouco travando valor ta boa,pouco travando valor ta boa,pouco travando valor ta boa,pouco travar valor to bom
4,5,"Vendedor confiável, produto ok e entrega antes...","[vendedor, confiavel, produto, ok, entrega, an...",vendedor confiável produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo,vendedor confiavel produto ok entregar antes p...
...,...,...,...,...,...,...,...
31469,5,Entregou dentro do prazo. O produto chegou em ...,"[entregou, dentro, prazo, produto, chegou, con...",entregou dentro prazo produto chegou condições...,entregou dentro prazo produto chegou condicoes...,entregou dentro prazo produto chegou condicoes...,entregar dentro prazo produto chegar condicoe ...
31470,3,"O produto não foi enviado com NF, não existe v...","[produto, nao, enviado, nf, nao, existe, venda...",produto não enviado nf não existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...,produto nao enviado em f nao existir venda em ...
31471,5,"Excelente mochila, entrega super rápida. Super...","[excelente, mochila, entrega, super, rapida, s...",excelente mochila entrega super rápida super r...,excelente mochila entrega super rapida super r...,excelente mochila entrega super rapida super r...,excelente mochila entregar super rapida super ...
31472,1,Solicitei a compra de uma capa de retrovisor c...,"[solicitei, compra, capa, retrovisor, celta, p...",solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...


In [105]:
df_lem.reset_index(drop=True, inplace=True)

In [106]:
df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in df_lem.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])],topn=5)
for idx, similaridade in frases_similares:
    print(f"Review: {df_lem.iloc[int(idx)]['tratamento_4']} - Similaridade: {similaridade:.4f}")

/tmp/ipython-input-3079940512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)


Review: entrega demorando - Similaridade: 0.8909
Review: comprei dois unidade apenas entregar - Similaridade: 0.8754
Review: vir faltando cadeira - Similaridade: 0.8729
Review: compra realizar dia 26 01 2018 hoje 26 02 2018 pruduto nao entregar - Similaridade: 0.8717
Review: ultimamente atrasar entrega - Similaridade: 0.8707


## Treinamento do Doc2vec

In [107]:
modelo = Doc2Vec(vector_size = 300, min_count = 2,
                 window=5, workers = 1, seed=45, epochs = 15)
modelo.build_vocab(dados_tag)

modelo.train(dados_tag, total_examples=modelo.corpus_count,
             epochs= modelo.epochs)

## Verificando a similardade - Doc2Vec

In [108]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])],topn=5)

for idx, similaridade in frases_similares:
    print(f"Review: {df_lem.iloc[int(idx)]['tratamento_1']} - Similaridade: {similaridade:.4f}")

Review: entrega atrasou - Similaridade: 0.9432
Review: comprei dois itens recebi apenas - Similaridade: 0.9409
Review: ultimamente atrasando entregas - Similaridade: 0.9370
Review: mt mboo - Similaridade: 0.9356
Review: voltaria comprar nesta loja - Similaridade: 0.9353


In [109]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['vendedor'])],topn=5)

for idx, similaridade in frases_similares:
    print(f"Review: {df_lem.iloc[int(idx)]['tratamento_1']} - Similaridade: {similaridade:.4f}")

Review: produto entregue recomendo vendedor - Similaridade: 0.9462
Review: vendedor não atualizava situação mercadoria gerou insegurança status entrega aguardando emissão nota fiscal - Similaridade: 0.9420
Review: ótimo vendedor superou expectativas produto excelente - Similaridade: 0.9366
Review: bom produto vendedor - Similaridade: 0.9344
Review: compraria novamente vendedor - Similaridade: 0.9343


In [110]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['reputacao loja'])],topn=5)

for idx, similaridade in frases_similares:
    print(f"Review: {df_lem.iloc[int(idx)]['tratamento_1']} - Similaridade: {similaridade:.4f}")

Review: bonito atende diz otimo produto - Similaridade: 0.1957
Review: produto atende proposito boa qualidade recebi antes prazo acordado - Similaridade: 0.1860
Review: agora produto atende propõe - Similaridade: 0.1758
Review: gostei produto atendeu vendedor dizia - Similaridade: 0.1741
Review: boa tarde comprei tres produtos so chegou dois nao responder produto chegou favor resolvam - Similaridade: 0.1723


# Aula 4 - Considerando o significado das frases

In [113]:
modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'

In [114]:
import tensorflow_hub as hun

modelo = hun.load(modulo_url)

In [115]:
modelo

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x79b9a8c60a40>

In [116]:
reviews = df['tratamento_3'].tolist()
reviews_emb = modelo(reviews)
reviews_emb

<tf.Tensor: shape=(31474, 512), dtype=float32, numpy=
array([[ 0.04775169,  0.01094663, -0.0541508 , ...,  0.09078011,
        -0.02599917, -0.09261213],
       [-0.00746649, -0.06605157, -0.04505714, ...,  0.05561579,
        -0.02624756, -0.07801282],
       [-0.00820429,  0.04282852, -0.02907449, ...,  0.07496072,
        -0.06071885, -0.07618523],
       ...,
       [ 0.02074313, -0.0543616 , -0.06415588, ...,  0.0719423 ,
         0.0105929 , -0.07603392],
       [-0.04131037, -0.03054746,  0.01322938, ...,  0.06785525,
        -0.0252889 , -0.07106064],
       [ 0.05717909, -0.05808825, -0.02038886, ...,  0.07339787,
        -0.06409648, -0.08014238]], dtype=float32)>

In [117]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(reviews_emb)

array([[1.0000002 , 0.61586297, 0.5175547 , ..., 0.5427756 , 0.57541025,
        0.6691027 ],
       [0.61586297, 0.99999994, 0.5666107 , ..., 0.63920605, 0.6011834 ,
        0.6711744 ],
       [0.5175547 , 0.5666107 , 0.9999999 , ..., 0.58997375, 0.6564975 ,
        0.5922985 ],
       ...,
       [0.5427756 , 0.63920605, 0.58997375, ..., 1.        , 0.6378722 ,
        0.61628115],
       [0.57541025, 0.6011834 , 0.6564975 , ..., 0.6378722 , 1.0000001 ,
        0.6389235 ],
       [0.6691027 , 0.6711744 , 0.5922985 , ..., 0.61628115, 0.6389235 ,
        1.        ]], dtype=float32)

## Avaliando as sentenças similares

In [122]:
import numpy as np

def sentencas_similares(tema, reviews, reviews_emb, top_n=5):
  tema_emb = modelo([tema])
  similaridades = cosine_similarity(tema_emb, reviews_emb).flatten()

  indices_similares = np.argsort(-similaridades)

  for idx in indices_similares[:top_n]:
    print(f"Review: {reviews[idx]} - Similaridade: {similaridades[idx]:.4f}")

In [123]:
sentencas_similares('entrega', reviews, reviews_emb)

Review: eficiente entrega - Similaridade: 0.6829
Review: entrega eficiente - Similaridade: 0.6756
Review: agil entrega - Similaridade: 0.6563
Review: entrega prazo - Similaridade: 0.6473
Review: entrega agil - Similaridade: 0.6471


In [125]:
sentencas_similares('reputação loja', reviews, reviews_emb, top_n=10)

Review: otima loja - Similaridade: 0.8578
Review: recomento loja - Similaridade: 0.8339
Review: pessima loja - Similaridade: 0.8335
Review: adorei loja - Similaridade: 0.8170
Review: rapidez eficiencia loja - Similaridade: 0.8018
Review: loja confianca - Similaridade: 0.7909
Review: produto loja confianca - Similaridade: 0.7888
Review: loja ruim - Similaridade: 0.7868
Review: otimas experiencias loja - Similaridade: 0.7839
Review: satisfeita produto loja - Similaridade: 0.7816


# Aula 5 - Transformando a similaridade em aplicação

In [112]:
nltk.download('punkt')
nltk.download('stopwords')

modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
